In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
%matplotlib inline

In [76]:
results = pd.read_csv('data/fifa/international_results.csv')
results['date'] = pd.to_datetime(results['date'], utc=True)

# restrict dates
earliest_date = '2010-01-01'
latest_date = '2018-06-14'
results = results[(results['date'] > earliest_date) & (results['date'] < latest_date)]
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
31700,2010-01-02 00:00:00+00:00,Iran,Korea DPR,1,0,Friendly,Doha,Qatar,True
31701,2010-01-02 00:00:00+00:00,Qatar,Mali,0,0,Friendly,Doha,Qatar,False
31702,2010-01-02 00:00:00+00:00,Syria,Zimbabwe,6,0,Friendly,Kuala Lumpur,Malaysia,True
31703,2010-01-02 00:00:00+00:00,Yemen,Tajikistan,0,1,Friendly,Sana'a,Yemen,False
31704,2010-01-03 00:00:00+00:00,Angola,Gambia,1,1,Friendly,Vila Real de Santo António,Portugal,True


In [77]:
ratings = pd.read_csv('data/sofifa_final.csv')
ratings['date'] = pd.to_datetime(ratings['date'])

# restrict dates
# note that we use a different earliest data here to ensure 
# that our earliest match data has a matching ratings data
ratings = ratings[(ratings['date'] > '2009-08-29') & (ratings['date'] < latest_date)]
ratings.head()

,team,date,overall,attack,midfield,defence,prestige,start_age,full_age,bup_speed,...,cc_passing,cc_crossing,cc_shooting,d_pressure,d_aggresion,d_width,wage_euros_thousands,value_euros_millions,growth,goalkeeeper_overall
48,Brazil,2018-06-11,85.0,86.0,83.0,85.0,10.0,26.73,27.09,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,123.608696,32.5,1.826087,84.0
49,Brazil,2018-06-07,85.0,86.0,83.0,85.0,10.0,26.73,27.09,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,123.608696,32.5,1.826087,84.0
50,Brazil,2018-06-04,85.0,86.0,83.0,85.0,10.0,26.73,27.09,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,123.608696,32.5,1.826087,84.0
51,Brazil,2018-05-31,85.0,86.0,83.0,85.0,10.0,26.73,27.09,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,123.608696,32.5,1.826087,84.0
52,Brazil,2018-05-28,85.0,86.0,83.0,85.0,10.0,26.73,27.09,51.0,...,39.0,33.0,67.0,76.0,73.0,63.0,123.608696,32.5,1.826087,84.0


We now have to find the closest match in date between our FIFA team ratings and the match data that we have.

In [78]:
dates = pd.Series(ratings.date.unique())
teams = ratings.team.unique()

In [79]:
# check that we found the corresponding team from team ratings in results
for t in teams:
    try:
        assert np.sum(results.home_team.unique() == t) == 1
    except:
        print('Cannot Find {}'.format(t))

for t in teams:
    try:
        assert np.sum(results.away_team.unique() == t) == 1
    except:
        print('Cannot Find {}'.format(t))

Cannot Find Republic of Ireland
Cannot Find United States
Cannot Find China PR
Cannot Find Côte d'Ivoire
Cannot Find Republic of Ireland
Cannot Find United States
Cannot Find China PR
Cannot Find Côte d'Ivoire


Before we proceed with anything else, let's check that country names are spelled similarly. We find that 'Republic of Ireland', 'United States', 'China PR', and 'Côte d'Ivoire' might be spelled differently.

In [80]:
ratings.replace('Republic of Ireland', 'Ireland', inplace=True)
ratings.replace('United States', 'USA', inplace=True)
ratings.replace('China PR', 'China', inplace=True)
ratings.replace("Côte d'Ivoire", 'Ivory Coast', inplace=True)

Now we finally have two datasets of teams spelled in the same way.

In [ ]:
# find closest date that we have data on
def get_latest_date(match_date, dates):
    match_date = match_date.to_datetime64()
    return dates[match_date >  dates].max()

# results['closest_date'] = results.apply(lambda i: min(dates, key=lambda d: abs(d-i.date)), axis=1)
results['closest_date'] = results['date'].apply(get_latest_date, dates = dates)
results.head()

Now that we have the closest matching data for both of our datasets, we can merge on team and date. Note that we have merge twice in order to account for both teams in the results table.

In [ ]:
# merge home team with their closest ratings
results_ratings1 = results.merge(ratings, how='inner', 
                                left_on=['closest_date', 'home_team'],
                                right_on=['date', 'team'])

results_ratings2 = results_ratings1.merge(ratings, how='inner',
                                       left_on=['closest_date', 'away_team'],
                                       right_on=['date', 'team'])

In [ ]:
results_ratings2.shape

We find that we have 1171 observations that can still be used after matching team ratings with the results dataframe with an inner merge. We now clean up the merged dataframe slightly.

In [ ]:
# response variable
score_diff = results_ratings2['home_score'] - results_ratings2['away_score']
results_ratings2['home_win'] = [0 if score < 0 else 1 if score > 0 else 2 for score in score_diff]

# drop useless columns
results_ratings2.drop(['home_team', 'away_team', 'tournament', 
                       'city', 'country', 'neutral', 'closest_date',
                      'team_x', 'date_y', 'team_y', 'date',
                      'home_score', 'away_score', 'date_x'], axis=1, inplace=True)

# reorder columns
results_ratings2.sort_index(axis=1, inplace=True)
results_ratings2.head()

Our final training data is basically a difference in the various team ratings. Now let's clean up the dataframe for our training.

In [ ]:
ratings_base = results_ratings2.loc[:,['home_win']]
ratings_base.head()

In [ ]:
diff_ratings = results_ratings2.drop('home_win', axis=1)
diff_ratings = diff_ratings.diff(axis=1)
ncol = diff_ratings.shape[1]

# we only want every alternate column
diff_ratings = diff_ratings.iloc[:,list(np.arange(1,ncol, 2))]

# we want our statistics to be from perspective of home team
diff_ratings = diff_ratings*-1
diff_ratings.head()

In [ ]:
columns = ['attack_diff', 'bup_dribbling_diff', 'bup_passing_diff', 'bup_speed_diff',
          'cc_crossing_diff', 'cc_passing_diff', 'cc_shooting_diff', 'd_aggresion_diff',
          'd_pressure_diff', 'd_width_diff', 'defence_diff', 'full_age_diff', 
          'goalkeeper_overall_diff', 'growth_diff', 'midfield_diff', 'overall_diff',
          'prestige_diff', 'start_age_diff', 'avg_value_euros_millions_diff',
          'avg_wage_euros_thousands_diff']

diff_ratings.columns = columns

# compile datframe
train = pd.concat([ratings_base, diff_ratings], axis=1)
train.head()

In [ ]:
print(train[train.avg_wage_euros_thousands_diff == 0].shape)
print(train[train.avg_value_euros_millions_diff == 0].shape)

Importantly, we note that 558 out of our 1171 observations have missing wage and value data.

In [ ]:
# save to csv
train.to_csv('data/train_team.csv', index = False)

We now create our test set with actual world cup data.

In [3]:
ratings_wc = pd.read_csv('data/sofifa_final.csv')
ratings_wc['date'] = pd.to_datetime(ratings_wc['date'], utc=True)

# restrict dates
latest_date = '2018-06-14'
wc_pre = '2018-06-14' # pre WC rating for values and wages
wc_start = '2018-06-16' # first WC rating
wc_end = '2018-07-15'

# get values and wages data
wage = ratings_wc.loc[ratings_wc['date'] == wc_pre, 'wage_euros_thousands']
value = ratings_wc.loc[ratings_wc['date'] == wc_pre, 'value_euros_millions']

# restrict dates
ratings_wc = ratings_wc[(ratings_wc['date'] >= wc_start) & (ratings_wc['date'] <= wc_end)]

# replace values and wage data
ratings_wc['wage_euros_thousands'] = wage.reset_index(drop=True)
ratings_wc['value_euros_millions'] = value.reset_index(drop=True)

ratings_wc.head()

NameError: name 'pd' is not defined

In [ ]:
dates_wc = ratings_wc.date.unique()
teams_wc = ratings_wc.team.unique()

In [ ]:
#https://gitlab.com/djh_or/2018-world-cup-stats/blob/master/world_cup_2018_stats.csv
results_wc = pd.read_csv("data/world_cup_2018_stats.csv")
results_wc.head()

Before we proceed, let's ensure that our country names match up similarly.

In [ ]:
# check that we found the corresponding team from team ratings in results
# but now we reverse it such that we try to find a rating for each wc team
for t in results_wc.Team.unique():
    try:
        assert np.sum(teams_wc == t) == 1
    except:
        print('Cannot Find {}'.format(t))


In [ ]:
ratings_wc.replace('Iran', 'IR Iran', inplace=True)

In [ ]:
results_wc = results_wc.loc[:,['Team', 'Opponent', 'WDL']]
results_wc['home_win'] = [0 if score == 'L' else 1 if score == 'W' else 2 for score in results_wc.WDL]
results_wc.drop('WDL', axis=1, inplace=True)

# we only want every alternate column
results_wc = results_wc.iloc[list(np.arange(0, results_wc.shape[0], 2)),:]

results_wc.head()

Now we can match the ratings data to our test dataset as well. We note that there might be some value in using the updated FIFA ratings at each time of the match. However, for simplicity, let's just assume that teams have the same rating throughout the tournament. We will take the rating at the start of the tournament.

In [ ]:
ratings_wc_start = ratings_wc[ratings_wc['date'] == wc_start]

# merge ratings with results table
results_ratings_wc1 = results_wc.merge(ratings_wc_start, how='left',
                                     left_on='Team', right_on='team')
results_ratings_wc2 = results_ratings_wc1.merge(ratings_wc_start, how='left',
                                             left_on='Opponent', right_on='team')



In [ ]:
# check that every team has a rating
assert results_ratings_wc2.shape[0] == results_wc.shape[0]

In [ ]:
# drop useless columns
results_ratings_wc2.drop(['Team', 'Opponent', 'team_x', 'team_y', 'date_y', 'date_x'], 
                         axis=1, inplace=True)

# reorder columns
results_ratings_wc2.sort_index(axis=1, inplace=True)
results_ratings_wc2.head()

In [ ]:
ratings_base_wc = results_ratings_wc2.loc[:,['home_win']]
ratings_base_wc.head()

In [ ]:
diff_ratings_wc = results_ratings_wc2.drop('home_win', axis=1)
diff_ratings_wc = diff_ratings_wc.diff(axis=1)
ncol_wc = diff_ratings_wc.shape[1]

# we only want every alternate column
diff_ratings_wc = diff_ratings_wc.iloc[:,list(np.arange(1,ncol_wc, 2))]

# we want our statistics to be from perspective of home teams
diff_ratings_wc = diff_ratings_wc*-1
diff_ratings_wc.head()

In [ ]:
diff_ratings_wc.columns = columns

# compile datframe
test = pd.concat([ratings_base_wc, diff_ratings_wc], axis=1)
test.head()

In [ ]:
# save to csv
test.to_csv('data/test_team.csv', index = False)